In [1]:
from pathlib import Path
import sys

PATH_FOR_PROJECT = str(Path().resolve().parent)
sys.path.append(PATH_FOR_PROJECT)

SMOKE_TEST = False
RANDOM_STATE_WITHOUT_TEMPERATURES = 32
RANDOM_STATE_WITH_TEMPERATURES = 18

# 1. Preparing the data

In [2]:
import seaborn as sns

In [3]:
iris = sns.load_dataset("iris")

x_n = iris.columns[:-1]
x_2 = iris[x_n].values

X = (x_2 - x_2.mean(axis=0)) / x_2.max(axis=0)

data_classes = list(iris['species'].unique())
y = iris['species'].apply(data_classes.index)

number_of_data = len(y)

# 2. Model without temperatures

In [4]:
from sklearn.metrics import accuracy_score

In [5]:
from uncertainty_forecasting.utils.timers import TimerContext
from uncertainty_forecasting.utils.time_profiler_logging import TimeProfilerLogger

In [6]:
from uncertainty_forecasting.models.classification.classification_bayesian_softmax_temperature import \
    BayesianSoftmaxClassificationWithTemperatures
from uncertainty_forecasting.models.classification.classification_bayesian_softmax import BayesianSoftmaxClassification
from uncertainty_forecasting.models.calibration.diagnostics import calculate_static_calibration_error

Can not use cuDNN on context None: cannot compile with cuDNN. We got this error:
b'/tmp/try_flags_q9jekbue.c:4:10: fatal error: cudnn.h: No such file or directory\n #include <cudnn.h>\n          ^~~~~~~~~\ncompilation terminated.\n'
ERROR (theano.gpuarray): Could not initialize pygpu, support disabled
Traceback (most recent call last):
  File "/home/louis/anaconda3/envs/env_UncertaintyForecasting/lib/python3.6/site-packages/theano/gpuarray/__init__.py", line 227, in <module>
    use(config.device)
  File "/home/louis/anaconda3/envs/env_UncertaintyForecasting/lib/python3.6/site-packages/theano/gpuarray/__init__.py", line 214, in use
    init_dev(device, preallocate=preallocate)
  File "/home/louis/anaconda3/envs/env_UncertaintyForecasting/lib/python3.6/site-packages/theano/gpuarray/__init__.py", line 140, in init_dev
    context.free_gmem / MB))
RuntimeError: Trying to preallocate 3843 MB of GPU memory while only 3117 MB are available.


In [7]:
from uncertainty_forecasting.data_handling.train_test_split import return_train_test_split_indexes

In [8]:
logger_time = TimeProfilerLogger.getInstance()

In [ ]:
with TimerContext(name="without_temperature",
                  show_time_when_exit=True,
                  logger=logger_time) as timer:

    train_indexes, test_indexes = return_train_test_split_indexes(number_of_data,
                                                                  test_size=0.3,
                                                                  random_state=RANDOM_STATE_WITHOUT_TEMPERATURES)

    X_train, X_test, y_train, y_test = X[train_indexes], X[test_indexes], \
                                       y.values[train_indexes], y.values[test_indexes]

    model_without = BayesianSoftmaxClassification(number_of_classes=3,
                                                  number_of_features=4,
                                                  X_train=X_train,
                                                  y_train=y_train)

    model_without.params.number_of_tuning_steps = 5000 if not SMOKE_TEST else 10
    model_without.params.number_of_samples_for_posterior = int(1e5) if not SMOKE_TEST else 10
    model_without.params.number_of_iterations = int(1e6) if not SMOKE_TEST else 10

    model_without.sample()
    #model_without.show_trace()

    predictions = model_without.make_predictions(X_test, y_test)

    y_test_predictions, confidences = predictions.predictions_with_confidence

    accuracy_without_temperatures = 100.0 * accuracy_score(y_test, predictions.predictions)

    curves_without_temperatures, means_per_bin_without_temperatures, deviation_score_without_temperatures = calculate_static_calibration_error(y_test_predictions,
                                                                                                y_test,
                                                                                                confidences,
                                                                                                predictions.number_of_classes)


Average Loss = 63.67:  64%|██████▍   | 643958/1000000 [11:02<05:48, 1020.82it/s] 

# 3. Model with temperatures

In [ ]:
with TimerContext(name="with_temperature", show_time_when_exit=True) as timer:

    train_indexes, test_indexes = return_train_test_split_indexes(number_of_data,
                                                                  test_size=0.3,
                                                                  random_state=RANDOM_STATE_WITH_TEMPERATURES)

    X_train, X_test, y_train, y_test = X[train_indexes], X[test_indexes], \
                                       y.values[train_indexes], y.values[test_indexes]

    model_with = BayesianSoftmaxClassificationWithTemperatures(number_of_classes=3,
                                                               number_of_features=4,
                                                               X_train=X_train,
                                                               y_train=y_train)

    model_with.params.number_of_tuning_steps = 5000 if not SMOKE_TEST else 10
    model_with.params.number_of_samples_for_posterior = int(1e5) if not SMOKE_TEST else 10
    model_with.params.number_of_iterations = int(1e6) if not SMOKE_TEST else 10

    model_with.sample()
    #model_with.show_trace()

    print(logger_time.times)

    predictions = model_with.make_predictions(X_test, y_test)

    y_test_predictions, confidences = predictions.predictions_with_confidence

    accuracy_with_temperatures = 100.0 * accuracy_score(y_test, predictions.predictions)

    curves_with_temperatures, means_per_bin_with_temperatures, deviation_score_with_temperatures = calculate_static_calibration_error(y_test_predictions,
                                                                                          y_test,
                                                                                          confidences,
                                                                                          predictions.number_of_classes)


# 4. Analysis of the results

In [ ]:
import pymc3 as pm
import numpy as np

In [ ]:
from uncertainty_forecasting.visualisations.visualisations import Visualisator

In [ ]:
model_with.model.name = 'with temperatures'
model_without.model.name = 'without temperatures'

df_compare_WAIC = pm.compare({model_with.model: model_with.trace,
                           model_without.model: model_without.trace})

print(df_compare_WAIC)


Visualisator.show_calibration_curves(means_per_bin_with_temperatures,
                                     np.array([curves_with_temperatures,
                                      curves_without_temperatures]),
                                     ["with temperatures", "without temperatures"],
                                     title_suffix="(SCE)"
                                     )

In [ ]:
print("acc without: {}".format(accuracy_without_temperatures))
print("acc with: {}".format(accuracy_with_temperatures))
print("dev without: {}".format(deviation_score_without_temperatures))
print("dev with: {}".format(deviation_score_with_temperatures))